In [49]:
# import libraries
import openai
import langchain
from pinecone import Pinecone as pc
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain.llms import OpenAI

In [50]:
from dotenv import load_dotenv
load_dotenv()

True

In [51]:
import os

In [52]:
# Let's ready the document
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

In [53]:
doc = read_doc("E:/langchain_llm_app/documents")
len(doc)

58

In [54]:
# Divide the docs into chunks
def chunk_data(docs, chunk_size=800, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    documents = text_splitter.split_documents(docs)
    return documents

In [ ]:
documents = chunk_data(doc)
documents

In [56]:
len(documents)

141

In [ ]:
# Embedding technique of OpenAI
embeddings = OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

In [58]:
vectors = embeddings.embed_query("How are you bro?")
len(vectors)

1536

In [59]:
# Vector Search DB in Pinecone
pc1 = pc(
    api_key=""
)

In [60]:
index_name="langchainvector"
index = Pinecone.from_documents(documents=documents, embedding=embeddings, index_name=index_name)

In [61]:
# Cosine similarity retrieve results from vectordb
def retrieve_query(query,k=2):
    matching_results = index.similarity_search(query, k=k)
    return matching_results

In [62]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [75]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5)

In [76]:
chain = load_qa_chain(llm, chain_type="stuff")

In [77]:
# Search answers from vectordb
def retrieve_answer(query):
    doc_search = retrieve_query(query)
    print(doc_search)
    response = chain.run(input_documents=doc_search, question=query)
    return response

In [ ]:
our_query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answer(our_query)
print(answer)